<a href="https://colab.research.google.com/github/ankit-rathi/Quantvesting_v2/blob/main/myStocks_Portfolio_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pyxirr
!pip install ta
!pip install yfinance==0.2.59

  Installing build dependencies ... -

 \

 done


  Getting requirements to build wheel ... -

 done


...

 done


 done
  Created wheel for ta: filename=ta-0.11.0-py3-none-any.whl size=29482 sha256=88c1c4ebd7afeb49253c3f0294185115929893f74ada5df483396dfe2bbfd419
  Stored in directory: /home/runner/.cache/pip/wheels/ec/b6/81/d781ff040ee9432384560f29596ee14ca79d70529d4228e50f
Successfully built ta


  Attempting uninstall: protobuf
    Found existing installation: protobuf 6.33.1
    Uninstalling protobuf-6.33.1:
      Successfully uninstalled protobuf-6.33.1


  Attempting uninstall: yfinance
    Found existing installation: yfinance 0.2.66
    Uninstalling yfinance-0.2.66:
      Successfully uninstalled yfinance-0.2.66
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 0/2 [protobuf]

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [yfinance]


In [2]:
# import libraries

import numpy as np
import pandas as pd
import datetime
import warnings
warnings.filterwarnings('ignore')
import yfinance as yfin
import ta
import matplotlib.pyplot as plt
import requests

In [3]:
# notebook start time
import time
import datetime
import pytz

start_time = time.time()

# function to print date timestamp
def print_date_time():
  tz_NY = pytz.timezone('Asia/Kolkata')
  datetime_NY = datetime.datetime.now(tz_NY)
  print("Run date time (IST):", datetime_NY.strftime("%Y-%m-%d %H:%M:%S"))

In [4]:
# function to get booked and reserved amount
def get_amt():
  # fetch the JSON data from the URL
  url = "https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioAmts.json"
  response = requests.get(url)
  pf_amts = response.json()  # parse the JSON data

  # extract values from the JSON
  py_booked_amt_dm = pf_amts["py_booked_amt_dm"]
  py_booked_amt_sv = pf_amts["py_booked_amt_sv"]
  cy_booked_amt_dm = pf_amts["cy_booked_amt_dm"]
  cy_booked_amt_sv = pf_amts["cy_booked_amt_sv"]
  reserve_amt_dm = pf_amts["reserve_amt_dm"]
  reserve_amt_sv = pf_amts["reserve_amt_sv"]

  # perform calculations
  py_booked_amt = py_booked_amt_dm + py_booked_amt_sv
  cy_booked_amt = cy_booked_amt_dm + cy_booked_amt_sv
  reserve_amt = reserve_amt_dm + reserve_amt_sv
  total_booked_amt = py_booked_amt + cy_booked_amt

  return total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt

gen_output = 0

In [5]:
# function to format the amount
def format_amt(number):
    abs_number = abs(number)

    if abs_number >= 1_00_00_000:  # Crores
        formatted_amt = f"{number / 1_00_00_000:.2f} C"
    elif abs_number >= 1_00_000:  # Lakhs
        formatted_amt = f"{number / 1_00_000:.2f} L"
    elif abs_number >= 1_000:  # Thousands
        formatted_amt = f"{number / 1_000:.2f} K"
    else:
        formatted_amt = f"{number:.2f}"

    return formatted_amt

# function to set start and end date
def get_start_end_date():
  start_date = (datetime.date.today() + datetime.timedelta(days=-365)).strftime('%Y-%m-%d')
  end_date = (datetime.date.today() + datetime.timedelta(days=1)).strftime('%Y-%m-%d')
  return start_date, end_date

# function to fetch my portfolio csv
def get_mypfs_df():
  mypfs_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioStocks.csv')
  return mypfs_df

# function to fetch my prospects csv
def get_mypps_df():
  mypps_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myProspectsScrips.csv')
  return mypps_df

# function to fetch screener data
def get_myscreen_df():
  myscreen_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myScreenerDB.csv')
  return myscreen_df

# function to fetch momentum data
def get_myinvmt_df():
  myinvmt_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myInvestments.csv')
  return myinvmt_df

# function to get the stock ids
def get_stock_ids(df_pf):
  stock_n100 = df_pf['Symbol'].unique()

  exclude = ['CADILAHC','MMTC', 'MASFIN']

  stock_ids = df_pf[~df_pf['Symbol'].isin(exclude) ]['Symbol'].unique()

  #mypf = mypf[mypf.Forecast.notnull()]
  #stock_ids = mypf['Symbol'].unique()

  stock_ids.sort()
  return stock_ids

# get features from screener data
def get_screener_features():
  myscreen_df = get_myscreen_df()
  cols = ['Symbol', 'EPS', 'MedPE', 'ROCE%', 'ROE%', 'CapType']
  return myscreen_df[cols]

# get relative strength
def get_relative_strength(stock_list):

    # Dictionary to store stock tickers and their corresponding percentage price change
    stock_changes = {}

    # Loop through each stock and fetch its price data
    for stock in stock_list:
        try:
            # Download the stock data for the given date range
            stock_data = yfin.Ticker(stock + '.NS').history(period='1mo', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

            # Calculate the percentage change for the stock
            if len(stock_data) > 0:
                start_price = stock_data['Close'].iloc[0]
                end_price = stock_data['Close'].iloc[-1]
                percent_change = round(((end_price - start_price) / start_price) * 100, 2)
                stock_changes[stock] = percent_change
            else:
                stock_changes[stock] = np.nan  # If no data is available, set to NaN

        except Exception as e:
            print(f"Error fetching data for {stock}: {e}")
            stock_changes[stock] = np.nan

    # Create a DataFrame with stock tickers and their percentage changes
    df = pd.DataFrame(list(stock_changes.items()), columns=['Symbol', 'Percent_Change'])

    # Drop any stocks with missing data (NaN values)
    df = df.dropna()

    # Calculate the percentile rank based on percentage change
    df['RSP'] = round(df['Percent_Change'].rank(pct=True) * 100, 2)
    cols = ['Symbol', 'RSP']
    df = df[cols]
    # Sort by percentile rank (optional)
    df = df.sort_values(by='RSP', ascending=False).reset_index(drop=True)

    return df


In [6]:
# function to get stock technicals
def stock_prec_dev(stock_symbol):
    #stock_symbol = 'ULTRACEMCO.NS'
    short_window = 20
    mid_window = 50
    long_window = 200
    moving_avg = 'SMA'

    start = datetime.datetime(*map(int, start_date.split('-')))
    end = datetime.datetime(*map(int, end_date.split('-')))

    stock_df = yfin.Ticker(stock_symbol + '.NS').history(period='max', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

    stock_df['Max'] = round(max(stock_df['Close']),2)
    stock_df = stock_df[(stock_df.index <= end_date) & (stock_df.index >= start_date)]
    stock_df['200_SMA'] = round(stock_df['Close'].rolling(window = 200, min_periods = 1).mean(),0)
    stock_df['Dev%_200'] = round((stock_df['Close'] - stock_df['200_SMA'])*100/stock_df['200_SMA'],2)
    stock_df.dropna(axis = 0, inplace = True) # remove any null rows

    stock_df['20_SMA'] = stock_df['Close'].rolling(window=20).mean()
    stock_df['50_SMA'] = stock_df['Close'].rolling(window=50).mean()
    stock_df['Symbol'] = stock_symbol

    stock_df['Close'] = round(stock_df['Close'],2)
    stock_df['Min'] = round(min(stock_df['Close']),2)
    stock_df['RSI_14'] = ta.momentum.RSIIndicator(close=stock_df['Close'], window=14).rsi()
    stock_df['RSI_14'] = round(stock_df['RSI_14'],0)
    stock_df['Prev_Close'] = stock_df['Close'].shift(1)
    stock_df.drop(['Open', 'Low', 'High', 'Volume'], axis=1, inplace=True)
    stock_df = stock_df.tail(1)

    max_SMA = max(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    min_SMA = min(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    ABS_Spread = max_SMA - min_SMA
    stock_df['Spread%'] = round((ABS_Spread / stock_df['200_SMA'].item()) * 100,2)

    return stock_df

# function to compute stock attributes
def get_common_features(stock_ids, df_mypf):

  df_prec_dev = pd.DataFrame()
  df_tmp = get_screener_features()
  df_rs = get_relative_strength(stock_ids)

  for stock_id in stock_ids:
      tmp = stock_prec_dev(stock_id)
      tmp = tmp.reset_index()
      df_prec_dev = pd.concat([df_prec_dev, tmp], ignore_index = True)
  df_prec_dev = pd.merge(df_prec_dev, df_mypf, on= 'Symbol')
  df_prec_dev = pd.merge(df_prec_dev, df_tmp, on= 'Symbol', how='left')
  df_prec_dev = pd.merge(df_prec_dev, df_rs, on= 'Symbol', how='left')
  #print(df_prec_dev.columns)
  df_prec_dev['Curr_PE'] = round(df_prec_dev['Close']/df_prec_dev['EPS'],1)
  df_prec_dev['Dev%_PE'] = round((df_prec_dev['Curr_PE'] - df_prec_dev['MedPE'])*100/df_prec_dev['MedPE'],2)
  df_prec_dev['Conviction'] = df_prec_dev['Conviction'] + '-' + df_prec_dev['CapType']
  return df_prec_dev

# function to arrange stock features
def arrange_features(df_stocks, common_cols, diff_cols):
  df_stocks_common = df_stocks[common_cols].drop_duplicates()
  df_stocks_diff = df_stocks[diff_cols]
  df_stocks_diff['Investment'] = df_stocks_diff['AvgCost'] * df_stocks_diff['Shares']
  df_stocks_diff = df_stocks_diff.groupby(['Symbol'])[['Shares', 'Investment']].aggregate(['sum']).reset_index()
  df_stocks_diff.columns = ['Symbol', 'Shares', 'Investment']
  df_stocks_diff['AvgCost'] = round(df_stocks_diff['Investment']/df_stocks_diff['Shares'],2)
  df_stocks = pd.merge(df_stocks_diff, df_stocks_common, on='Symbol')
  return df_stocks

# plot fact distribution across dimension
def plot_pie_chart(df, dimension, fact):
  # grouping the data by category and calculating the sum of fact for each type
  grouped_data = df.groupby(dimension)[fact].sum()

  # sorting the grouped data in descending order
  grouped_data = grouped_data.sort_values(ascending=False)

  # creating a pie chart
  grouped_data.plot.pie(autopct='%1.1f%%', startangle=90, figsize=(6, 6))

  # adding a title and displaying the plot
  plt.title(f'{dimension} {fact} Distribution')
  plt.ylabel('')  # To hide the y-label
  plt.show()

In [7]:
# function to get portfolio features

def get_portfolio_features(df_common_features):

  print_date_time()
  print('-------------------')

  #df_common_features["Target"] = df_common_features["Target"].fillna(df_common_features["Max"])
  df_common_features['NTT'] = np.where(df_common_features['Strategy']== 'NTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['LTT'] = np.where(df_common_features['Strategy']== 'BTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['BOL'] = df_common_features['Min']

  tmp_df = df_common_features[df_common_features['Symbol'].isin(mypf_df[mypf_df['InPortfolio'] != 'NA'].Symbol.values)]
  print('qualified stocks: '+str(len(tmp_df['Symbol'].unique())))
  tmp_df1 = tmp_df[tmp_df['LatestQtr'] == 1]
  print('with latest results: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df1 = tmp_df1[tmp_df1['StarStock'] == 1]
  print('still star stocks: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df['Investment'] = tmp_df['AvgCost'] * tmp_df['Shares']
  tmp_df['Current'] = round(tmp_df['Close'] * tmp_df['Shares'],0)
  tmp_df['Previous'] = tmp_df['Prev_Close'] * tmp_df['Shares']
  tmp_df['EstimatedST'] = tmp_df['NTT'] * tmp_df['Shares']
  tmp_df['EstimatedLT'] = tmp_df['LTT'] * tmp_df['Shares']
  tmp_df['Current P/L'] = round((tmp_df['Current'] - tmp_df['Investment']),0)
  tmp_df['Today P/L%'] = round((tmp_df['Current'] - tmp_df['Previous'])*100/tmp_df['Previous'],2)
  tmp_df['Current P/L%'] = round((tmp_df['Current'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedST P/L%'] = round((tmp_df['EstimatedST'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedLT P/L%'] = round((tmp_df['EstimatedLT'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['NTT%'] = round((tmp_df['NTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['LTT%'] = round((tmp_df['LTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['Gained%'] = round((tmp_df['Close'] - tmp_df['BOL'])*100/tmp_df['BOL'],2)
  investment = round(sum(tmp_df['AvgCost']*tmp_df['Shares']),0)
  current = round(sum(tmp_df['Close']*tmp_df['Shares']),0)
  tmp_df['InitAlloc%'] = round(tmp_df['Investment']*100/investment,2)
  tmp_df['CurrAlloc%'] = round(tmp_df['Current']*100/current,2)
  tmp_df['FTT'] = tmp_df['LTT']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT'] = tmp_df['NTT']
  tmp_df['FTT%'] = tmp_df['LTT%']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT%'] = tmp_df['NTT%']
  tmp_df['FTT Amt'] = round(tmp_df['FTT%'] * tmp_df['Current']/100,0)
  tmp_df['OTT%'] = round((tmp_df['FTT'] - tmp_df['AvgCost'])*100/tmp_df['AvgCost'],2)
  tmp_df['RRR Ind'] = round(tmp_df['Current P/L']/tmp_df['FTT Amt'],2)
  tmp_df['Risk Ind'] = round(tmp_df['Current P/L%']*tmp_df['CurrAlloc%'],0)

  return tmp_df

# function to print portfolio features
def print_portfolio_stats(df_portfolio_features, myinvmt_df):
  from pyxirr import xirr

  total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt = get_amt()

  dates = myinvmt_df['Date'].values
  dates = np.append(dates, datetime.date.today().strftime('%d-%b-%y'))
  investment = myinvmt_df['Investment'].values
  dates= pd.to_datetime(dates)

  current = round(sum(df_portfolio_features['Close']*df_portfolio_features['Shares']),0) + reserve_amt
  investment_xirr = np.append(investment, current)
  cagr = round(xirr(pd.DataFrame({"dates": dates, "amounts": investment_xirr}))*100,2)

  investment = -sum(investment)
  invested = round(sum(df_portfolio_features['AvgCost']*df_portfolio_features['Shares']),0) + reserve_amt
  previous = round(sum(df_portfolio_features['Prev_Close']*df_portfolio_features['Shares']),0) + reserve_amt
  cy_invested = investment + py_booked_amt

  today_pnl_amount = current-previous
  today_pnl_percentage = round((current-previous)*100/previous,2)

  curr_pnl_amount = current-invested
  curr_pnl_percentage = round((curr_pnl_amount)*100/(cy_invested),2)

  cy_pnl_amount = cy_booked_amt + curr_pnl_amount
  cy_pnl_percentage = round((cy_pnl_amount)*100/cy_invested,2)

  overall_pnl_amount = total_booked_amt + curr_pnl_amount
  overall_pnl_percentage = round((overall_pnl_amount)*100/investment,2)

  estimate_st = round(sum(df_portfolio_features['FTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_st_pnl_amount = estimate_st-current
  est_st_pnl_percentage = round((est_st_pnl_amount)*100/current,2)

  estimate_lt = round(sum(df_portfolio_features['LTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_lt_pnl_amount = estimate_lt-current
  est_lt_pnl_percentage = round((est_lt_pnl_amount)*100/current,2)

  total_profit = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Investment']),0)
  total_loss = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Investment']),0)

  cy_booked_percentage = round((cy_booked_amt)*100/current,2)
  py_booked_percentage = round((py_booked_amt)*100/investment,2)
  total_booked_percentage = round((total_booked_amt)*100/investment,2)

  print('-------------------')
  print('Initial Investment: ', format_amt(investment))
  print('CY Investment: ', format_amt(cy_invested))
  print('Reserve: ', format_amt(reserve_amt))
  print('Current: ',  format_amt(current))
  print('-------------------')
  print('Today PnL: '+ '{} ({}%)'.format(format_amt(today_pnl_amount), today_pnl_percentage))
  print('Current PnL: '+ '{} ({}%)'.format(format_amt(curr_pnl_amount), curr_pnl_percentage))
  print('CY Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(cy_pnl_amount), cy_pnl_percentage))
  print('-------------------')
  print('Total profit: ', format_amt(total_profit))
  print('Total loss: ', format_amt(total_loss))
  print('-------------------')
  print('Total Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(overall_pnl_amount), overall_pnl_percentage))
  print('Total Booked PnL: '+ '{} ({}%)'.format(format_amt(total_booked_amt), total_booked_percentage))
  print('Curr Year Booked PnL: '+ '{} ({}%)'.format(format_amt(cy_booked_amt), cy_booked_percentage))
  print('Prev Year Booked PnL: '+ '{} ({}%)'.format(format_amt(py_booked_amt), py_booked_percentage))
  print('===================')
  print('Est FTT: ',  format_amt(estimate_st))
  print('Est FTT PnL: '+ '{} ({}%)'.format(format_amt(est_st_pnl_amount), est_st_pnl_percentage))

  print('===================')
  print('Deployed: ', format_amt(investment))

  print('Current: ', format_amt(current))

  print('CAGR/XIRR %: '+'{}%'.format(cagr))

In [8]:
# get start and end date
start_date, end_date = get_start_end_date()

# get portfolio and prospects data
mypfs_df = get_mypfs_df()
mypps_df = get_mypps_df()
myinvmt_df = get_myinvmt_df()

# merge above datasets
mypf_df = pd.merge(mypfs_df, mypps_df, on="Symbol")

# seggregate dm and sv portfolio
dm_pf = mypf_df[mypf_df['InPortfolio'] == 'DM']
sv_pf = mypf_df[mypf_df['InPortfolio'] == 'SV']

# build portfolio stock dataframe
dm_stocks = get_stock_ids(dm_pf)
sv_stocks = get_stock_ids(sv_pf)
df_stocks = pd.concat([dm_pf,sv_pf], ignore_index = True)

# arrange common and diff stock features
common_cols = ['Symbol', 'Target', 'Criteria', 'Strategy', 'CumlRnk', 'LatestQtr', 'StarStock', 'Conviction', 'Category', 'InFolio', 'XIRR', 'MBQ']
diff_cols = ['Symbol', 'AvgCost', 'Shares']
df_stocks = arrange_features(df_stocks, common_cols, diff_cols)

# get common features
stock_ids = df_stocks['Symbol'].values
df_common_features = get_common_features(stock_ids, df_stocks)

df_common_features.reset_index(drop=True, inplace=True)
df_common_features.drop(['Date'], axis=1, inplace=True)
# get and print portfolio features
df_portfolio_features = get_portfolio_features(df_common_features)

print_portfolio_stats(df_portfolio_features, myinvmt_df)

df = df_portfolio_features
#plot_pie_chart(df, 'CapType', 'Current')
list_ox40n = list(df[df['MBQ'].str.contains('OX40N', na=False)]['Symbol'].values)

Run date time (IST): 2025-12-04 16:10:52
-------------------
qualified stocks: 86
with latest results: 25
still star stocks: 15
-------------------
Initial Investment:  1.28 C
CY Investment:  1.55 C
Reserve:  4.95 L
Current:  1.44 C
-------------------
Today PnL: 16.74 K (0.12%)
Current PnL: -24.34 L (-15.72%)
CY Booked + Current PnL: -10.05 L (-6.49%)
-------------------
Total profit:  1.36 L
Total loss:  -25.70 L
-------------------
Total Booked + Current PnL: 16.93 L (13.24%)
Total Booked PnL: 41.27 L (32.29%)
Curr Year Booked PnL: 14.29 L (9.93%)
Prev Year Booked PnL: 26.98 L (21.11%)
Est FTT:  2.33 C
Est FTT PnL: 88.97 L (61.81%)
Deployed:  1.28 C
Current:  1.44 C
CAGR/XIRR %: 6.76%


In [9]:
# top 5 near their targets
cols = ['Symbol', 'FTT','Today P/L%', 'Current P/L%', 'FTT%', 'OTT%', 'FTT Amt', 'Current P/L', 'Current', 'Dev%_PE', 'RSI_14','Conviction', 'Spread%',  'CumlRnk', 'RRR Ind', 'CurrAlloc%', 'Gained%', 'Criteria', 'Strategy', 'Category']
df_tmp = df_portfolio_features[~df_portfolio_features['Symbol'].isin(['ENRIN','BLUSPRING','DIGITIDE'])]
df_tmp = df_tmp.sort_values(by = 'FTT Amt', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
39,INDIGOPNTS,1408.00,1.12,-7.65,8.31,0.02,13396.0,-13359.0,161200.0,146.09,68.0,M-SC,15.57,234.0,-1.00,1.16,40.22,OX40N,NTT,PAINTS
77,TTKPRESTIG,770.00,-0.08,-16.77,20.27,0.09,17001.0,-16904.0,83873.0,94.66,37.0,M-SC,2.75,253.0,-0.99,0.60,10.19,OX40N,NTT,DURABLES
43,ITC,452.00,0.64,-1.52,12.14,10.44,23927.0,-3047.0,197091.0,-41.15,45.0,X-LC,1.57,5.0,-0.13,1.42,3.96,X40,NTT,FMCG
35,ICICIGI,2252.93,0.54,5.79,13.61,20.19,25099.0,10099.0,184419.0,-17.20,48.0,X-MC,6.62,68.0,0.40,1.33,21.78,X40,ATH,INSURANCE
50,MASFIN,398.61,-3.47,-5.40,29.02,22.05,26897.0,-5295.0,92685.0,-18.25,48.0,H-SC,6.82,164.0,-0.20,0.67,34.72,XR,ATH,FINANCE


In [10]:
# top 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
80,VALIANTORG,838.00,4.99,-44.28,212.57,74.15,193766.0,-72451.0,91154.0,-264.07,43.0,H-SC,19.99,149.0,-0.37,0.66,16.84,XR,NTT,CHEMICALS
13,BSOFT,831.70,3.11,-20.97,90.80,50.79,100928.0,-29499.0,111154.0,1.28,75.0,H-SC,5.54,151.0,-0.29,0.80,29.42,XR,ATH,IT
29,HAPPSTMNDS,1480.71,2.80,-36.20,189.74,84.85,155147.0,-46397.0,81768.0,-24.12,56.0,H-SC,14.00,147.0,-0.30,0.59,4.94,AR,ATH,IT
15,CAMS,4762.00,2.52,3.69,20.35,24.79,53142.0,9286.0,261142.0,-1.02,56.0,X-SC,2.16,86.0,0.17,1.88,27.68,X40N,NTT,MISC
26,GILLETTE,11206.78,2.44,0.85,31.30,32.42,80144.0,2154.0,256050.0,-16.99,46.0,X-SC,8.39,85.0,0.03,1.84,16.08,X40,ATH,FMCG


In [11]:
# bottom 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
50,MASFIN,398.61,-3.47,-5.40,29.02,22.05,26897.0,-5295.0,92685.0,-18.25,48.0,H-SC,6.82,164.0,-0.20,0.67,34.72,XR,ATH,FINANCE
44,JCHAC,2282.00,-3.21,-33.58,50.54,-0.01,42903.0,-42915.0,84890.0,17269.57,8.0,M-SC,0.97,233.0,-1.00,0.61,0.00,OX40N,NTT,AC
18,COFFEEDAY,80.00,-2.31,-43.49,139.38,35.27,89435.0,-49383.0,64166.0,-55.73,25.0,L-SC,14.16,270.0,-0.55,0.46,57.05,XR,NTT,HOTELS
69,SURYODAY,216.00,-2.30,-21.58,56.90,23.03,79901.0,-38648.0,140423.0,53.51,43.0,H-SC,10.04,167.0,-0.48,1.01,39.33,XR,NTT,BANKS
57,RAJOOENG,143.10,-2.22,-32.61,107.18,39.61,74029.0,-33430.0,69070.0,-54.49,28.0,H-SC,23.45,135.0,-0.45,0.50,0.00,AR,ATH,MISC


In [12]:
# top 5 to exit based on CumlRnk
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > -1) & (df_tmp['Current P/L%'] < 1)].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
37,IEX,219.0,-0.64,0.39,48.04,48.63,96436.0,787.0,200741.0,-33.16,63.0,H-SC,14.73,136.0,0.01,1.44,11.99,XR,NTT,MISC


In [13]:
# OX40N stocks to exit
df_tmp = df_portfolio_features[(df_portfolio_features['Symbol'].isin(list_ox40n))].sort_values(by = 'Current P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
25,FINCABLES,1641.55,-0.15,-6.19,121.64,107.91,168474.0,-9142.0,138502.0,-23.79,32.0,M-SC,11.32,216.0,-0.05,1.00,0.00,OX40N,ATH,CABLES
39,INDIGOPNTS,1408.00,1.12,-7.65,8.31,0.02,13396.0,-13359.0,161200.0,146.09,68.0,M-SC,15.57,234.0,-1.00,1.16,40.22,OX40N,NTT,PAINTS
77,TTKPRESTIG,770.00,-0.08,-16.77,20.27,0.09,17001.0,-16904.0,83873.0,94.66,37.0,M-SC,2.75,253.0,-0.99,0.60,10.19,OX40N,NTT,DURABLES
47,KANSAINER,340.00,-1.29,-23.35,48.04,13.47,99296.0,-62973.0,206694.0,-68.51,33.0,H-SC,2.37,158.0,-0.63,1.49,5.10,XY24,NTT,PAINTS
66,SIS,528.00,0.32,-24.22,60.02,21.26,50499.0,-26895.0,84137.0,1984.97,48.0,H-SC,4.04,166.0,-0.53,0.61,13.68,OX40N,NTT,MISC


In [14]:
# top 5 to book for rotation
df_tmp = df_portfolio_features[(df_portfolio_features['Current P/L%'] > 20) ].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category


In [15]:
# top 5 to monitor for exit
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > 0) ].sort_values(by = 'Dev%_PE', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
84,WIPRO,420.0,0.88,5.38,63.47,72.27,113336.0,9118.0,178566.0,-9.71,69.0,M-LC,3.07,99.0,0.08,1.28,11.89,XR,NTT,IT
37,IEX,219.0,-0.64,0.39,48.04,48.63,96436.0,787.0,200741.0,-33.16,63.0,H-SC,14.73,136.0,0.01,1.44,11.99,XR,NTT,MISC


In [16]:
# top 5 to book for rotation from weak conviction
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[~df_tmp['Criteria'].isin(['XY25','XY24','X40', 'X40N'])].sort_values(by = 'RRR Ind', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
84,WIPRO,420.00,0.88,5.38,63.47,72.27,113336.0,9118.0,178566.0,-9.71,69.0,M-LC,3.07,99.0,0.08,1.28,11.89,XR,NTT,IT
37,IEX,219.00,-0.64,0.39,48.04,48.63,96436.0,787.0,200741.0,-33.16,63.0,H-SC,14.73,136.0,0.01,1.44,11.99,XR,NTT,MISC
38,INDIAMART,4810.62,-1.60,-1.49,109.86,106.72,133471.0,-1844.0,121492.0,-51.20,35.0,H-SC,1.55,139.0,-0.01,0.87,21.53,AR,ATH,MISC
85,ZYDUSLIFE,1286.17,0.09,-1.32,37.45,35.63,77447.0,-2773.0,206801.0,-15.41,42.0,H-MC,3.41,119.0,-0.04,1.49,15.13,AR,ATH,PHARMA
25,FINCABLES,1641.55,-0.15,-6.19,121.64,107.91,168474.0,-9142.0,138502.0,-23.79,32.0,M-SC,11.32,216.0,-0.05,1.00,0.00,OX40N,ATH,CABLES


In [17]:
# top 5 to accumulate based on lowest RSI_14
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'RSI_14', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
9,BAJAJHFL,181.50,-0.68,-21.23,88.45,48.45,139705.0,-42558.0,157948.0,-26.96,12.0,X-MC,9.42,64.0,-0.30,1.14,0.00,X40N,ATH,FINANCE
11,BATAINDIA,2096.00,-1.75,-38.05,118.27,35.20,94267.0,-48965.0,79705.0,4.55,23.0,X-SC,14.43,92.0,-0.52,0.57,0.00,X40,NTT,FOOTWEAR
22,DIXON,18931.72,-2.22,-11.97,38.12,21.58,47026.0,-16776.0,123363.0,-55.63,25.0,X-MC,5.75,56.0,-0.36,0.89,9.22,X40N,ATH,IT
23,DMART,5391.45,0.09,-7.01,37.77,28.12,73903.0,-14742.0,195665.0,-19.77,30.0,X-LC,4.21,19.0,-0.20,1.41,14.96,X40N,ATH,FMCG
46,JSWINFRA,342.00,-0.63,-9.55,27.40,15.23,49650.0,-19143.0,181204.0,-22.54,30.0,X-MC,6.80,66.0,-0.39,1.30,19.61,X40N,NTT,REALTY


In [18]:
# top 5 to accumulate based on Spread and CumlRnk
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['CumlRnk'] < 100)].sort_values(by = 'Spread%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
3,ACC,3906.0,-1.36,-23.72,115.18,64.15,209075.0,-56431.0,181520.0,-55.44,41.0,X-SC,1.49,82.0,-0.27,1.31,1.82,XY24,BTT,CEMENT
8,AWL,485.0,0.69,-21.12,94.58,53.50,225133.0,-63717.0,238034.0,-61.84,31.0,X-MC,1.55,58.0,-0.28,1.71,5.73,XY24,NTT,FMCG
43,ITC,452.0,0.64,-1.52,12.14,10.44,23927.0,-3047.0,197091.0,-41.15,45.0,X-LC,1.57,5.0,-0.13,1.42,3.96,X40,NTT,FMCG
15,CAMS,4762.0,2.52,3.69,20.35,24.79,53142.0,9286.0,261142.0,-1.02,56.0,X-SC,2.16,86.0,0.17,1.88,27.68,X40N,NTT,MISC
36,ICICIPRULI,790.0,0.67,2.01,28.38,30.97,51169.0,3560.0,180298.0,-21.95,51.0,X-MC,2.22,75.0,0.07,1.30,14.83,X40,ATH,INSURANCE


In [19]:
# top 5 to accumulate based on lowest Gained%
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'Gained%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
11,BATAINDIA,2096.00,-1.75,-38.05,118.27,35.20,94267.0,-48965.0,79705.0,4.55,23.0,X-SC,14.43,92.0,-0.52,0.57,0.00,X40,NTT,FOOTWEAR
9,BAJAJHFL,181.50,-0.68,-21.23,88.45,48.45,139705.0,-42558.0,157948.0,-26.96,12.0,X-MC,9.42,64.0,-0.30,1.14,0.00,X40N,ATH,FINANCE
55,QUESS,424.00,-1.37,-31.08,107.24,42.84,48049.0,-20201.0,44805.0,-54.82,35.0,X-SC,37.33,83.0,-0.42,0.32,0.00,XY24,NTT,MISC
54,PGHH,17907.65,0.25,-5.36,41.33,33.76,78554.0,-10755.0,190065.0,-33.18,32.0,X-MC,4.67,57.0,-0.14,1.37,0.25,X40,ATH,FMCG
53,PAGEIND,51605.07,0.71,-8.38,37.60,26.07,56408.0,-13720.0,150020.0,-32.53,32.0,X-MC,10.70,55.0,-0.24,1.08,0.71,X40,ATH,APPARELS


In [20]:
# top 5 to accumulate based on lowest CurrAlloc%
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
55,QUESS,424.00,-1.37,-31.08,107.24,42.84,48049.0,-20201.0,44805.0,-54.82,35.0,X-SC,37.33,83.0,-0.42,0.32,0.00,XY24,NTT,MISC
59,RELAXO,1176.00,0.51,-47.13,188.09,52.31,144344.0,-68418.0,76742.0,-43.76,38.0,X-SC,5.34,91.0,-0.47,0.55,2.72,X40N,NTT,FOOTWEAR
11,BATAINDIA,2096.00,-1.75,-38.05,118.27,35.20,94267.0,-48965.0,79705.0,4.55,23.0,X-SC,14.43,92.0,-0.52,0.57,0.00,X40,NTT,FOOTWEAR
22,DIXON,18931.72,-2.22,-11.97,38.12,21.58,47026.0,-16776.0,123363.0,-55.63,25.0,X-MC,5.75,56.0,-0.36,0.89,9.22,X40N,ATH,IT
51,MEDANTA,1486.00,-0.96,0.26,23.75,24.08,30230.0,335.0,127285.0,-7.89,39.0,X-SC,5.92,89.0,0.01,0.92,19.93,XY24,NTT,HEALTHCARE


In [21]:
# top 5 to accumulate based on CumlRnk
df_tmp = df_portfolio_features.sort_values(by = 'CumlRnk', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
73,TCS,4389.97,1.55,-12.26,35.95,19.28,109124.0,-42411.0,303545.0,-23.62,69.0,X-LC,6.53,1.0,-0.39,2.18,12.22,X40,ATH,IT
41,INFY,2275.00,1.20,8.76,42.40,54.87,144283.0,27401.0,340289.0,-15.29,67.0,X-LC,2.98,2.0,0.19,2.45,17.77,X40,BTT,IT
75,TMPV,600.00,-0.11,-15.53,68.33,42.18,159533.0,-42935.0,233475.0,-24.30,32.0,X-LC,3.55,3.0,-0.27,1.68,1.13,XY24,NTT,AUTO
81,VBL,671.64,0.50,-3.06,39.95,35.67,122317.0,-9666.0,306176.0,-14.52,60.0,X-LC,5.31,4.0,-0.08,2.20,10.37,X40N,ATH,FMCG
43,ITC,452.00,0.64,-1.52,12.14,10.44,23927.0,-3047.0,197091.0,-41.15,45.0,X-LC,1.57,5.0,-0.13,1.42,3.96,X40,NTT,FMCG


In [22]:
# top 5 for average up
df_tmp = df_portfolio_features[(df_portfolio_features['Dev%_200'] > 5)].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
40,INDUSINDBK,1800.00,1.90,-36.29,108.57,32.89,53407.0,-28016.0,49191.0,-707.41,64.0,L-MC,6.21,259.0,-0.52,0.35,35.47,XR,NTT,BANKS
6,ASIANTILES,137.00,-0.83,-15.78,115.71,81.67,91418.0,-14804.0,79006.0,7100.00,42.0,L-SC,16.37,271.0,-0.16,0.57,53.00,XR,NTT,CERAMICS
50,MASFIN,398.61,-3.47,-5.40,29.02,22.05,26897.0,-5295.0,92685.0,-18.25,48.0,H-SC,6.82,164.0,-0.20,0.67,34.72,XR,ATH,FINANCE
13,BSOFT,831.70,3.11,-20.97,90.80,50.79,100928.0,-29499.0,111154.0,1.28,75.0,H-SC,5.54,151.0,-0.29,0.80,29.42,XR,ATH,IT
69,SURYODAY,216.00,-2.30,-21.58,56.90,23.03,79901.0,-38648.0,140423.0,53.51,43.0,H-SC,10.04,167.0,-0.48,1.01,39.33,XR,NTT,BANKS


In [23]:
# top 5 RSP
df_tmp = df_portfolio_features.sort_values(by = 'RSP', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
39,INDIGOPNTS,1408.00,1.12,-7.65,8.31,0.02,13396.0,-13359.0,161200.0,146.09,68.0,M-SC,15.57,234.0,-1.00,1.16,40.22,OX40N,NTT,PAINTS
5,ASIANPAINT,3460.25,0.13,3.36,17.01,20.95,44266.0,8466.0,260234.0,2.32,79.0,X-LC,17.93,31.0,0.19,1.87,40.20,X40,ATH,PAINTS
13,BSOFT,831.70,3.11,-20.97,90.80,50.79,100928.0,-29499.0,111154.0,1.28,75.0,H-SC,5.54,151.0,-0.29,0.80,29.42,XR,ATH,IT
40,INDUSINDBK,1800.00,1.90,-36.29,108.57,32.89,53407.0,-28016.0,49191.0,-707.41,64.0,L-MC,6.21,259.0,-0.52,0.35,35.47,XR,NTT,BANKS
41,INFY,2275.00,1.20,8.76,42.40,54.87,144283.0,27401.0,340289.0,-15.29,67.0,X-LC,2.98,2.0,0.19,2.45,17.77,X40,BTT,IT


In [24]:
# Top N allocation
df_tmp = df_portfolio_features.sort_values(by = 'CurrAlloc%', ascending=False)
top_n_values = [10, 25, 50]

sum_df = pd.DataFrame({
    'Top_N': top_n_values,
    'Sum_Alloc%': [df_tmp['CurrAlloc%'].head(n).sum() for n in top_n_values]
})

sum_df

,Top_N,Sum_Alloc%
0,10,20.42
1,25,45.32
2,50,76.23


In [25]:
# market-cap-wise allocation
df_tmp = df_portfolio_features[cols]
df_tmp.groupby(df_tmp['Conviction'].str[-2:])['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
SC    46.16
MC    29.52
LC    24.31
Name: CurrAlloc%, dtype: float64

In [26]:
# criteria-wise allocation
df_portfolio_features.groupby('Criteria')['CurrAlloc%'].sum().sort_values(ascending=False)

Criteria
XY24     27.98
X40      21.94
X40N     12.14
XR        9.89
AR        9.27
XY25      9.24
OX40N     7.70
SR        1.03
MH        0.80
Name: CurrAlloc%, dtype: float64

In [27]:
# conviction-wise allocation
df_portfolio_features.groupby('Conviction')['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
H-SC    24.29
X-MC    22.91
X-LC    20.81
M-SC    12.09
X-SC     8.39
H-MC     4.86
M-MC     1.40
L-SC     1.39
M-LC     1.28
H-LC     1.21
L-LC     1.01
L-MC     0.35
Name: CurrAlloc%, dtype: float64

In [28]:
# sector-wise stats
df_tmp = df_portfolio_features.groupby('Category')[['CurrAlloc%', 'Current', 'Current P/L', 'FTT Amt']].sum().sort_values(by=['Current', 'Current P/L'], ascending=False)
df_tmp['Current P/L%'] = round(df_tmp['Current P/L'] * 100 / df_tmp['Current'], 2)
df_tmp['FTT%'] = round(df_tmp['FTT Amt'] * 100 / df_tmp['Current'], 2)
cols = ['CurrAlloc%', 'Current P/L%', 'FTT%']
df_tmp[cols].sort_values(by=['CurrAlloc%'], ascending=False)

,CurrAlloc%,Current P/L%,FTT%
Category,,,
FMCG,17.12,-5.74,39.93
IT,12.52,-15.74,78.32
FINANCE,9.45,-18.15,68.07
MISC,7.59,-23.09,73.51
ELECTRICAL,6.17,-9.54,49.66
PAINTS,6.14,-8.20,24.27
INSURANCE,4.45,-2.35,38.42
PHARMA,3.98,-1.23,33.63
AUTO,3.41,-21.33,73.35


In [29]:
# money to be made criteria-wise
df_portfolio_features.groupby('Criteria')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Criteria,,
XY24,3146918.0,21
XR,1286472.0,13
AR,1284906.0,10
X40,990849.0,14
X40N,760587.0,9
OX40N,721099.0,10
XY25,356867.0,6
SR,276614.0,2
MH,72804.0,1


In [30]:
# money to be made conviction-wise
df_portfolio_features.groupby('Conviction')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Conviction,,
H-SC,3601410.0,25
M-SC,1377948.0,15
X-MC,1300037.0,16
X-LC,869672.0,11
X-SC,753276.0,8
H-MC,404021.0,3
L-SC,268118.0,3
M-LC,113336.0,1
H-LC,65451.0,1


In [31]:
# money to be made criteria and conviction-wise
df_portfolio_features.groupby(['Conviction', 'Criteria'])['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

sum  count
Conviction Criteria                  
H-SC       XY24      1261343.0      6
           AR         886175.0      5
M-SC       XY24       786436.0      6
H-SC       XR         770660.0      7
X-MC       X40        452859.0      7
           XY24       385349.0      3
X-LC       X40        363579.0      5
H-SC       OX40N      333814.0      4
M-SC       OX40N      300020.0      5
X-SC       X40N       291511.0      3
           XY24       287354.0      3
H-SC       SR         276614.0      2
X-MC       X40N       272856.0      4
X-LC       XY24       232419.0      2
H-MC       AR         210004.0      2
X-LC       X40N       196220.0      2
H-MC       XY24       194017.0      1
X-MC       XY25       188973.0      2
L-SC       XR         180853.0      2
X-SC       X40        174411.0      2
M-SC       XR         168216.0      2
           AR         123276.0      2
M-LC       XR         113336.0      1
L-SC       OX40N       87265.0      1
X-LC       XY25        77454.0      2
H-SC       MH          72804.0      1
H-LC       AR          65451.0      1
L-MC       XR          53407.0      1
M-MC       XY25        51297.0      1
L-LC       XY25        39143.0      1

In [32]:
# notebook execution time

end_time = time.time()
execution_time = round(end_time - start_time, 0)
print(f"Notebook execution time: {execution_time} seconds")

Notebook execution time: 25.0 seconds
